# Example 2: _Mount Kosciuszko region_

In this second notebook, we illustrate the use of **bioLEC** to estimate the connectivity in different parts of the _Kosciuszko National Park_ region in Australia. 

**Mount Kosciuszko** is Australia's highest mountain at 2228 metres. It is located on the Main Range of the **Snowy Mountains** and is part of the _Australian Alps National Parks and Reserves_, in **New South Wales**.

![Region](https://github.com/Geodels/bioLEC/blob/master/src/bioLEC/Notebooks/images/kosinit.jpg?raw=true)


<div class="alert alert-block alert-info">
Here we use the Elevation Information System aka ELVIS a Geoscience Australia webportal design to access elevation datasets in an open, free and easily accessible environment. 
</div>

The dataset is made available from [**ELVIS webserver**](http://elevation.fsdf.org.au). Here we resampled the dataset to a 100 m resolution grid. The digital elevation grid file is found in `dataset/kosciuszko.csv`.


### Dependencies

In addition to bioLEC, this notebook relies on:
+ [lavavu](https://github.com/OKaluza/LavaVu) 
+ `pandas`.

In [ ]:
import lavavu
import numpy as np
import pandas as pd
import bioLEC as bLEC

%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

from pylab import rcParams
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.interpolate import RegularGridInterpolator
from mpl_toolkits.axes_grid1 import make_axes_locatable

%matplotlib inline

## A. Sub-regions definition 

We plot the entire regions and will then create some smaller regions that will be used to compute the LEC...

In [ ]:
df = pd.read_csv('../dataset/kosciuszko.csv', sep=' ', engine='c', header=None, na_filter=False, dtype=np.float,
                               low_memory=False)
XYZ = df.values
dx = XYZ[1,0]-XYZ[0,0]
nx = int((XYZ[:,0].max()-XYZ[:,0].min())/dx+1)
ny = int((XYZ[:,1].max()-XYZ[:,1].min())/dx+1)

In [ ]:
lv = lavavu.Viewer(border=False, background="#FFFFFF", resolution=[800,500], near=-10.0)

lvQuad = lv.quads("Z",  vertices=XYZ, dims=[nx,ny], wireframe=False, colour="#161616", opacity=1.0)
lvQuad.vertices(XYZ)
lvQuad.values(XYZ[:,2],"elevation")

cm = lvQuad.colourmap("dem1", range=[0,2700.], reverse=False)
cb = lvQuad.colourbar()
lvQuad.control.Panel()

# Obtained from lv.camera()
lv.translation(-650.607, 6108.765, -62771.094)
lv.rotation(-29.367, -1.062, -0.807)
lv.scale('z', 3)

lv.control.ObjectList()
lv.control.Range(command='scale z', range=(1,10), step=1, value=8)
lv.control.show()

We now define the 3 sub-regions...

In [ ]:
def regionGrid(Z, mind=0, maxd=2500, color=plt.cm.terrain, listRect=None):

    rcParams['figure.figsize'] = (9,9)
    rcParams['font.size'] = 8
    ax=plt.gca()
    im = ax.imshow(Z,interpolation='nearest',cmap=color,vmin=mind, vmax=maxd)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2%", pad=0.2)
    plt.colorbar(im,cax=cax)
    
    # Add the patch to the Axes
    if listRect is not None:
        for k in range(len(listRect)):
            ax.add_patch(listRect[k])
    plt.show()
    plt.close()

    return

In [ ]:
Z = np.flipud(XYZ[:,2].reshape(ny,nx))

# Create sub-regions
regions = []

# Sub-region 1
rect1 = patches.Rectangle((100,50),150,100,linewidth=1,edgecolor='k',facecolor=(0,0,0,0.25))
regions.append(rect1)
Z1 = np.flipud(Z[50:150,100:250]).reshape(150,100)

# Sub-region 2
rect2 = patches.Rectangle((120,200),150,150,linewidth=1,edgecolor='k',facecolor=(0,0,0,0.25))
regions.append(rect2)
Z2 = np.flipud(Z[200:350,120:270]).reshape(150,150)

# Sub-region 3
rect3 = patches.Rectangle((300,270),130,130,linewidth=1,edgecolor='k',facecolor=(0,0,0,0.25))
regions.append(rect3)
Z3 = np.flipud(Z[270:400,300:430]).reshape(130,130)

regionGrid(Z, color=plt.cm.terrain, listRect=regions)

## B. bioLEC class initialisation

<div class="alert alert-block alert-info">
Initialization function for building landscape elevational connectivity.
</div>

We will perform the **LEC** calculation on the sub-regions and obtain similar results as the ones presented in the figure below:

![Results](https://github.com/Geodels/bioLEC/blob/master/src/bioLEC/Notebooks/images/kosres.jpg?raw=true)

### Arguments

The filename needs to be provided without extension.

+ filename (`str`): `CSV` file name containing regularly spaced elevation grid [default: None]
+ XYZ (`3D Numpy Array`): 3D coordinates array of shape (nn,3) where nn is the number of points [default: None]
+ Z (`2D Numpy Array`): Elevation array of shape (nx,ny) where nx and ny are the number of points  along the X and Y axis [default: None]
+ dx (`float`): grid spacing in metre when the Z argument defined above is used [default: None]
+ periodic (`bool`):  applied periodic boundary to the elevation grid [default: False]
+ symmetric (`bool`): applied symmetric boundary to the elevation grid [default: False]
+ sigmap (`float`): species niche width percentage  based on elevation extent [default: 0.1]
+ sigmav (`float`): species niche fixed width values [default: None]
+ diagonals (`bool`): computes the path based on the diagonal moves as well as the axial ones eg. D4/D8 connectivity [default: True]
+ delimiter (`str`):  elevation grid csv delimiter [default: ' ']
+ sl (`float`):  sea level position used to remove marine points from the LEC calculation [default: -1.e6]

Here we use for input Z (`2D Numpy Array`) argument to initialise the main function. 

In [ ]:
#LEC1 = bLEC.landscapeConnectivity(Z=Z1, dx=100) 
LEC2 = bLEC.landscapeConnectivity(Z=Z2, dx=100) 
#LEC3 = bLEC.landscapeConnectivity(Z=Z3, dx=100) 

### Visualising the initial mesh

Define mesh coordinates from the package:

In [ ]:
# v1 = np.vstack([LEC1.X, LEC1.Y])
# v1 = np.vstack([v1, LEC1.Z]).T

v2 = np.vstack([LEC2.X, LEC2.Y])
v2 = np.vstack([v2, LEC2.Z]).T

# v3 = np.vstack([LEC3.X, LEC3.Y])
# v3 = np.vstack([v3, LEC3.Z]).T

In [ ]:
lv = lavavu.Viewer(border=False, background="#FFFFFF", resolution=[500,500], near=-10.0)

lvQuad = lv.quads("Z",  vertices=v2, dims=[LEC2.nx,LEC2.ny], wireframe=False, colour="#161616", opacity=1.0)
lvQuad.vertices(v2)
lvQuad.values(LEC2.Z,"elevation")

cm = lvQuad.colourmap("dem1", range=[800,2700.], reverse=False)
cb = lvQuad.colourbar()
lvQuad.control.Panel()

# Obtained from lv.camera()
lv.translation(272.886, -32.86, -24118.826)
lv.rotation(-13.641, 2.619, -0.509)
lv.scale('z', 3)

lv.control.ObjectList()
lv.control.Range(command='scale z', range=(1,10), step=1, value=8)
lv.control.show()

## C. Computing landscape elevational connectivity

This function computes the **minimum path for all nodes** in a given surface and **measure of the closeness** of each node to other at similar elevation range.

It then provide the *landscape elevational connectivity* array from computed measure of closeness calculation.

In [ ]:
#LEC1.computeLEC()
LEC2.computeLEC()
#LEC3.computeLEC()

### Visualisation of landscape elevational map

In [ ]:
vLEC = np.divide(LEC2.LEC.flatten(),LEC2.LEC.max())

lv1 = lavavu.Viewer(border=False, background="#FFFFFF", resolution=[500,500], near=-10.0)

lvQuad1 = lv1.quads("Z",  vertices=v2, dims=[LEC2.nx,LEC2.ny], wireframe=False, colour="#161616", opacity=1.0)
lvQuad1.vertices(v2)
lvQuad1.values(LEC2.Z,"elevation")
cm = lvQuad1.colourmap("dem1", range=[800,2700.], reverse=False)
cb = lvQuad1.colourbar()
lvQuad1.control.Panel()

lvQuad2 = lv1.quads("LEC",  vertices=v2, dims=[LEC2.nx,LEC2.ny], wireframe=False, colour="#161616", opacity=1.0)
lvQuad2.values(vLEC,"LEC")
cm1 = lvQuad2.colourmap("coolwarm", range=[0,1.], reverse=False)
cb1 = lvQuad2.colourbar()

# Obtained from lv.camera()
lv.translation(272.886, -32.86, -24118.826)
lv.rotation(-13.641, 2.619, -0.509)
lv.scale('z', 3)

lv1.control.ObjectList()
lv1.control.Range(command='scale z', range=(1,10), step=1, value=8)
lv1.control.show()

## D. Write LEC data

This function writes the computed landscape elevational connectivity array in a **CSV file**
and create a **VTK visualisation file** (.vts).

### Argument

+ filename (str): output file name without format extension.

<div class="alert alert-block alert-danger">
The filename needs to be provided without extension.
</div>

In [ ]:
#LEC1.writeLEC('region1-result')
LEC2.writeLEC('region2-result')
#LEC3.writeLEC('region3-result')

## E. Plotting elevation and LEC distribution

### Elevation & LEC maps

In [ ]:
#LEC1.viewResult(imName='plot1.png')
LEC2.viewResult(imName='plot2.png')
#LEC3.viewResult(imName='plot3.png')

The following functions plot and save in a figure the **distribution of LEC and elevation with elevation** in different flavours...


### Elevation frequency as a function of site elevation

In [ ]:
#LEC1.viewElevFrequency(input='region1-result', imName='elev1_freq.png', dpi=300)
LEC2.viewElevFrequency(input='region2-result', imName='elev2_freq.png', dpi=300)
#LEC3.viewElevFrequency(input='region3-result', imName='elev3_freq.png', dpi=300)

### LEC as a function of site elevation

In [ ]:
#LEC1.viewLECFrequency(input='region1-result', imName='lec1_freq.png', dpi=300)
LEC2.viewLECFrequency(input='region2-result', imName='lec2_freq.png', dpi=300)
#LEC3.viewLECFrequency(input='region3-result', imName='lec3_freq.png', dpi=300)

### Elevation frequency and LEC as a function of site elevation with error bar

In [ ]:
#LEC1.viewLECZbar(input='region1-result', imName='lec1_bar.png', dpi=300)
LEC2.viewLECZbar(input='region2-result', imName='lec2_bar.png', dpi=300)
#LEC3.viewLECZbar(input='region3-result', imName='lec3_bar.png', dpi=300)